# 第二十章 Tkinter 進階元件與畫布
---
本章延伸 Tkinter 的 GUI 設計：單選/複選按鈕、容器元件（Frame/LabelFrame/Toplevel）、訊息/對話盒、文字區域與捲軸、下拉式元件（OptionMenu/Combobox）、字型設定、以及 **Canvas** 畫圖、事件、動畫與小專題示例。

### ⚠️ 在 Colab 執行 Tkinter 的注意事項
Colab 沒有圖形介面 (headless)，直接使用 `tk.Tk()` 會報錯。

若要在 Colab 測試 Tkinter 程式：
1. 先執行下方**安裝**指令，建立虛擬顯示環境。
2. 本章各範例已加入 **RUN_GUI** 防呆：本機會顯示 GUI；Colab 會以虛擬顯示執行並提示。

In [ ]:
!apt-get -y update >/dev/null
!apt-get -y install xvfb python3-tk >/dev/null
!pip -q install pyvirtualdisplay pillow matplotlib

In [ ]:
import os
RUN_GUI = not ('COLAB_GPU' in os.environ)

if not RUN_GUI:
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(800, 600))
    display.start()
    print("已啟動 Colab 虛擬顯示環境（看不到視窗，但可執行 Tk 程式）")
else:
    print("偵測到本機環境，Tkinter 將正常顯示 GUI 視窗。")

> 下列範例皆假設已 `import tkinter as tk`，並使用 `root = tk.Tk()` 建立主視窗；
程式碼已包成 `if RUN_GUI: ... else: ...`，避免在 Colab 報錯。

## 20-1 選項鈕（Radiobutton）
**用途**：提供**單選**；多個選項綁定同一個 `tk.<Type>Var()` 變數，彼此互斥。

**語法**：
- 建立：`tk.Radiobutton(parent, text=文字, value=值, variable=變數, command=函式)`
- 常見參數：`text`、`value`、`variable`、`indicatoron=0`（用**方塊**取代圓鈕）、`image`/`compound`（圖文）


In [ ]:
import tkinter as tk

if RUN_GUI:
    root = tk.Tk(); root.title('Radiobutton 範例')
    choice = tk.StringVar(value='A')

    def on_pick():
        lbl.config(text=f"你選了：{choice.get()}")

    opts = {'A':'選項A','B':'選項B','C':'選項C'}
    for i,(val,text) in enumerate(opts.items()):
        tk.Radiobutton(root, text=text, value=val, variable=choice, command=on_pick).grid(row=0, column=i, padx=6)

    # 以盒子呈現
    tk.Radiobutton(root, text='盒子樣式', value='box', variable=choice, indicatoron=0, command=on_pick).grid(row=1, column=0, columnspan=2, sticky='ew', pady=6)

    lbl = tk.Label(root, text='請選擇')
    lbl.grid(row=2, column=0, columnspan=3, pady=8)

    root.mainloop()
else:
    print('Radiobutton 範例：單選、indicatoron=0 可改成盒子。')

## 20-2 核取按鈕（Checkbutton）
**用途**：提供**複選**。

**語法**： `tk.Checkbutton(parent, text=文字, variable=變數, onvalue=1, offvalue=0, command=函式)`

In [ ]:
import tkinter as tk

if RUN_GUI:
    root = tk.Tk(); root.title('Checkbutton 範例')
    flags = {'游泳': tk.IntVar(value=0), '閱讀': tk.IntVar(value=1), '程式': tk.IntVar(value=1)}

    def report():
        picked = [k for k,v in flags.items() if v.get()==1]
        lbl.config(text=f"興趣：{', '.join(picked) if picked else '（無）'}")

    for i,(text,var) in enumerate(flags.items()):
        tk.Checkbutton(root, text=text, variable=var, command=report).grid(row=0, column=i, padx=6)
    lbl = tk.Label(root, text='請勾選興趣'); lbl.grid(row=1, column=0, columnspan=3, pady=8)
    root.mainloop()
else:
    print('Checkbutton 範例：複選，使用 onvalue/offvalue 與 IntVar。')

## 20-3 容器元件：Frame / LabelFrame / Toplevel
- `tk.Frame(parent, **options)`：一般容器；可搭配 `relief='raised/sunken/groove/ridge'` 改變邊框樣式。
- `tk.LabelFrame(parent, text='標題', **options)`：帶標題的容器。
- `tk.Toplevel(parent)`：**獨立視窗**（有自己的標題列、可關閉）。

In [ ]:
import tkinter as tk

if RUN_GUI:
    root = tk.Tk(); root.title('容器元件')

    frm = tk.Frame(root, relief='groove', borderwidth=2)
    frm.pack(padx=10, pady=10, fill='x')
    tk.Label(frm, text='在 Frame 裡的 Label').pack(padx=6, pady=6)

    lfrm = tk.LabelFrame(root, text='設定')
    lfrm.pack(padx=10, pady=10, fill='x')
    tk.Checkbutton(lfrm, text='啟用通知').pack(anchor='w', padx=8, pady=4)

    def open_top():
        top = tk.Toplevel(root); top.title('Toplevel 視窗')
        tk.Label(top, text='這是獨立視窗').pack(padx=10, pady=10)

    tk.Button(root, text='開啟 Toplevel', command=open_top).pack(pady=6)
    root.mainloop()
else:
    print('Frame/LabelFrame/Toplevel：容器與獨立視窗示例。')

## 20-4 訊息（Message）與對話盒（messagebox）
- `tk.Message(parent, text='內容', width=寬度)`：自動換行的短訊息。
- `from tkinter import messagebox` → `messagebox.showinfo/showwarning/showerror/askyesno/...`
  - 常見參數：`title`、`message`、`icon`（`info/error/question/warning`）

In [ ]:
import tkinter as tk
from tkinter import messagebox

if RUN_GUI:
    root = tk.Tk(); root.title('Message / messagebox')
    tk.Message(root, text='這是一段會自動換行的訊息，適合顯示較長說明。', width=220).pack(padx=10, pady=10)
    def greet():
        messagebox.showinfo(title='My Message Box', message='Python tkinter 早安')
    tk.Button(root, text='Good Morning', command=greet).pack(pady=6)
    root.mainloop()
else:
    print('Message / messagebox：顯示說明文字與彈出對話框。')

## 20-5 文字區域（Text）與捲軸（Scrollbar）、下拉式元件（OptionMenu / Combobox）
- `tk.Text(parent, wrap='char/word/none')`：多行文字區域；可搭配 `Scrollbar`。
- `tk.Scrollbar(parent, orient='vertical/horizontal')`：與 `Text` 透過 `yscrollcommand` / `command` 連動。
- `tk.OptionMenu(parent, variable, *values)`：簡單下拉；
- `ttk.Combobox(parent, textvariable=var, values=tuple)`：輸入 + 下拉（需 `from tkinter import ttk`）。

In [ ]:
import tkinter as tk
from tkinter import ttk

if RUN_GUI:
    root = tk.Tk(); root.title('Text / Scrollbar / OptionMenu / Combobox')

    # Text + Scrollbar
    txt = tk.Text(root, height=6, wrap='word')
    scr = tk.Scrollbar(root, command=txt.yview)
    txt.configure(yscrollcommand=scr.set)
    txt.grid(row=0, column=0, sticky='nsew')
    scr.grid(row=0, column=1, sticky='ns')
    root.grid_rowconfigure(0, weight=1)
    root.grid_columnconfigure(0, weight=1)
    txt.insert('end', '多行文字...\n' * 10)

    # OptionMenu
    opt = tk.StringVar(value='請選擇')
    tk.OptionMenu(root, opt, '紅色', '綠色', '藍色').grid(row=1, column=0, sticky='w', pady=6)

    # Combobox
    cvar = tk.StringVar()
    combo = ttk.Combobox(root, textvariable=cvar, values=('Cat','Dog','Bird'))
    combo.current(0)
    combo.grid(row=2, column=0, sticky='w')

    def on_combo(_):
        tk.Label(root, text=f"選到：{cvar.get()}").grid(row=3, column=0, sticky='w')
    combo.bind('<<ComboboxSelected>>', on_combo)

    root.mainloop()
else:
    print('Text/Scrollbar/OptionMenu/Combobox：多行輸入與下拉元件示例。')

## 20-6 文字區域字型（`tkinter.font.Font`）
**用途**：建立**命名字型**以重複使用。

**語法**：`Font(root=None, font=None, name=None, exists=False, **options)`
- 常用 `options`：`family`、`size`、`weight`（`'bold'`）、`slant`（`'italic'`）

In [ ]:
import tkinter as tk
import tkinter.font as tkfont

if RUN_GUI:
    root = tk.Tk(); root.title('Font 範例')
    f = tkfont.Font(family='Arial', size=16, weight='bold')
    tk.Label(root, text='粗體 16pt 樣式', font=f).pack(padx=10, pady=10)
    root.mainloop()
else:
    print('Font：建立可重用的命名字型（family/size/weight 等）。')

## 20-7 畫布（Canvas）基礎繪圖
**建立**：`tk.Canvas(parent, width=寬, height=高, bg='背景')` → `.pack()` / `.grid()` 加入視窗。

**常用方法**：
- `create_line(x1, y1, x2, y2, ..., width, fill, dash, arrow, arrowshape=(d1,d2,d3))`
- `create_rectangle(x1, y1, x2, y2, width, fill, outline, dash)`
- `create_oval(x0, y0, x1, y1, ...)`（畫圓/橢圓；`start`/`extent` 可畫弧；`style='arc/chord/pieslice'`）
- `create_polygon(points, ...)`、`create_text(x, y, text=...)`、`create_image(x, y, image=...)`

In [ ]:
import tkinter as tk

if RUN_GUI:
    root = tk.Tk(); root.title('Canvas 基礎')
    cvs = tk.Canvas(root, width=360, height=240, bg='white'); cvs.pack(padx=6, pady=6)
    # 線條
    cvs.create_line(10, 10, 350, 10, width=2, fill='blue', arrow=tk.LAST, arrowshape=(10,12,6))
    # 矩形
    cvs.create_rectangle(20, 30, 120, 100, width=2, outline='green', fill='lightgreen')
    # 圓/弧
    cvs.create_oval(150, 30, 230, 110, fill='pink', outline='red')
    cvs.create_oval(250, 30, 330, 110, style='arc', start=30, extent=280, width=3)
    # 多邊形與文字
    cvs.create_polygon(40,160, 80,200, 20,200, fill='gold', outline='orange')
    cvs.create_text(220, 170, text='Hello Canvas', fill='purple')
    root.mainloop()
else:
    print('Canvas：line/rectangle/oval/polygon/text 基礎繪圖。')

### 📋 常用 Canvas 方法速查表
以下整理 Tkinter `Canvas` 常用繪圖方法與用途：

| 方法 | 用法範例 | 說明 |
|------|-----------|------|
| `create_line` | `cvs.create_line(x1,y1,x2,y2, ..., width=2, fill='red', dash=(4,2))` | 畫直線/折線，可設定粗細、顏色、虛線、箭頭 |
| `create_rectangle` | `cvs.create_rectangle(x0,y0,x1,y1, fill='blue')` | 畫矩形，左上角 (x0,y0)、右下角 (x1,y1) |
| `create_oval` | `cvs.create_oval(x0,y0,x1,y1, fill='green')` | 畫橢圓或圓，外接矩形為 (x0,y0)-(x1,y1) |
| `create_arc` | `cvs.create_oval(..., start=30, extent=120, style='arc')` | 畫弧形，可指定起始角度與範圍 |
| `create_polygon` | `cvs.create_polygon(x1,y1, x2,y2, ..., fill='yellow')` | 畫多邊形，點座標依序給定 |
| `create_text` | `cvs.create_text(x,y, text='Hello', font=('Arial',14), fill='black')` | 在指定座標顯示文字 |
| `create_image` | `cvs.create_image(x,y, image=tkimg, anchor='nw')` | 顯示影像（需 `PhotoImage` 或 `ImageTk.PhotoImage`） |
| `coords` | `cvs.coords(item_id, x0,y0,x1,y1)` | 讀取或設定物件的新座標 |
| `move` | `cvs.move(item_id, dx, dy)` | 將物件平移 dx, dy |
| `delete` | `cvs.delete(item_id)` 或 `cvs.delete('all')` | 刪除物件或清空畫布 |

> 提示：`create_xxx` 會回傳 **item_id**，可用於後續移動、修改、刪除。

### 在 Canvas 插入影像（建議用 Pillow）
`tk.PhotoImage` 支援格式有限；建議使用 `Pillow`：
```python
from PIL import Image, ImageTk
img = Image.open('path/to/img.png')
tkimg = ImageTk.PhotoImage(img)
canvas.create_image(x, y, image=tkimg, anchor='nw')
```

In [ ]:
import tkinter as tk
from PIL import Image, ImageDraw, ImageTk

if RUN_GUI:
    root = tk.Tk(); root.title('Canvas 影像')
    cvs = tk.Canvas(root, width=240, height=160, bg='white'); cvs.pack(padx=6, pady=6)
    # 動態建立一張示意圖片
    img = Image.new('RGB', (120, 80), 'lightblue')
    draw = ImageDraw.Draw(img)
    draw.text((10, 30), 'Pillow', fill='navy')
    tkimg = ImageTk.PhotoImage(img)
    cvs.create_image(10, 10, image=tkimg, anchor='nw')
    root.mainloop()
else:
    print('Canvas + Pillow：ImageTk.PhotoImage 載入影像並顯示。')

## 20-8 以滑鼠書寫（事件綁定）
- 綁定：`widget.bind('<B1-Motion>', handler)`（按住左鍵拖曳）
- 在處理函式讀取 `event.x`, `event.y` 取得座標，使用 `create_oval()` 畫小點形成線條。

In [ ]:
import tkinter as tk

if RUN_GUI:
    root = tk.Tk(); root.title('滑鼠書寫')
    cvs = tk.Canvas(root, width=300, height=200, bg='white'); cvs.pack(padx=6, pady=6)
    r = 2
    def paint(e):
        cvs.create_oval(e.x-r, e.y-r, e.x+r, e.y+r, fill='black', outline='')
    cvs.bind('<B1-Motion>', paint)
    root.mainloop()
else:
    print('以滑鼠書寫：<B1-Motion> 綁定 + create_oval 畫點。')

## 20-9 基本動畫與鍵盤控制
- 位移：`canvas.move(item_id, dx, dy)`、`canvas.update()` 或 `root.after(ms, func)`
- 鍵盤：`root.bind_all('<Key>', handler)`，於 `handler(event)` 中讀取 `event.keysym` 判斷方向。

In [ ]:
import tkinter as tk
import time

if RUN_GUI:
    root = tk.Tk(); root.title('動畫與鍵盤')
    cvs = tk.Canvas(root, width=320, height=200, bg='white'); cvs.pack(padx=6, pady=6)
    ball = cvs.create_oval(10, 90, 30, 110, fill='steelblue')

    # 簡單自動移動
    for _ in range(20):
        cvs.move(ball, 5, 0)
        cvs.update(); time.sleep(0.03)

    # 鍵盤控制
    step = 10
    def on_key(e):
        d = {'Left':(-step,0), 'Right':(step,0), 'Up':(0,-step), 'Down':(0,step)}
        if e.keysym in d:
            dx,dy = d[e.keysym]; cvs.move(ball, dx, dy)
    root.bind_all('<Key>', on_key)

    root.mainloop()
else:
    print('動畫 move() / update()，鍵盤 event.keysym 控制方向。')

## 20-10 小專題：繪製長條圖（排序與重置）
**目標**：隨機產生 1~20 的 20 筆資料，以長條圖顯示；提供「排序」「重置」按鈕，排序完成顯示對話盒。

In [ ]:
import tkinter as tk
from tkinter import messagebox
import random

if RUN_GUI:
    root = tk.Tk(); root.title('簡易長條圖')
    W,H = 420, 240
    cvs = tk.Canvas(root, width=W, height=H, bg='ivory'); cvs.pack(padx=8, pady=8)

    data = [random.randint(1,20) for _ in range(20)]

    def draw(vals):
        cvs.delete('all')
        w = W/len(vals)
        for i,v in enumerate(vals):
            x0 = i*w+4; y0 = H-10
            x1 = (i+1)*w-4; y1 = H-10 - v*8
            cvs.create_rectangle(x0, y1, x1, y0, fill='skyblue', outline='steelblue')
        cvs.update()

    def do_sort():
        data.sort()
        draw(data)
        messagebox.showinfo('完成', '排序完成！')

    def reset():
        nonlocal_data = [random.randint(1,20) for _ in range(20)]
        data[:] = nonlocal_data
        draw(data)

    btns = tk.Frame(root); btns.pack(pady=6)
    tk.Button(btns, text='排序', command=do_sort).pack(side='left', padx=6)
    tk.Button(btns, text='重置', command=reset).pack(side='left', padx=6)

    draw(data)
    root.mainloop()
else:
    print('長條圖小專題：排序/重置 + messagebox 通知。')

## 作業 / 挑戰
1. 使用 `LabelFrame` 與 `Radiobutton` 設計一組「主題配色」面板，切換時即時改變主視窗背景色。
2. 讓「滑鼠書寫」支援顏色挑選與筆刷粗細（`Scale` + `ColorChooser`）。
3. 將長條圖改為**動態排序動畫**（每一步交換都重繪），並新增「資料量」與「速度」控制。